# Lexicographical comparisons

## Lexicographical `<` uses `==`

Thus lexicographical comparisons work when elements are totally ordered but not for arbitrary weak orderings.

In [1]:
import logging

In [2]:
class Noisy:
    """A wrapper that logs rich comparisons and hashing."""
    
    __slots__ = ('_value',)
    
    def __init__(self, value):
        """Create a new Noisy wrapper for a given object."""
        self._value = value
    
    def __repr__(self):
        """Representation of this object for debugging."""
        return f'{type(self).__name__}({self._value})'
    
    def __str__(self):
        """Informal string representation, same as the wrapped object's."""
        return str(self._value)
    
    def __eq__(self, other):
        """Delegate to equality comparison for the wrapped object."""
        logging.info(f'{self!r}.__eq__({other!r})')
        
        if isinstance(other, type(self)):
            if hasattr(type(self._value), '__eq__'):
                return self._value.__eq__(other._value)
        
        return NotImplemented
    
    def __ne__(self, other):
        """Delegate to not-equal comparison for the wrapped object."""
        logging.info(f'{self!r}.__ne__({other!r})')
        
        if isinstance(other, type(self)):
            if hasattr(type(self._value), '__ne__'):
                return self._value.__ne__(other._value)
        
        return NotImplemented
    
    def __lt__(self, other):
        """Delegate to less-than comparison for the wrapped object."""
        logging.info(f'{self!r}.__lt__({other!r})')
        
        if isinstance(other, type(self)):
            if hasattr(type(self._value), '__lt__'):
                return self._value.__lt__(other._value)
        
        return NotImplemented
    
    def __gt__(self, other):
        """Delegate to greater-than comparison for the wrapped object."""
        logging.info(f'{self!r}.__gt__({other!r})')
        
        if isinstance(other, type(self)):
            if hasattr(type(self._value), '__gt__'):
                return self._value.__gt__(other._value)
        
        return NotImplemented

    def __le__(self, other):
        """Delegate to less-or-equal comparison for the wrapped object."""
        logging.info(f'{self!r}.__le__({other!r})')
        
        if isinstance(other, type(self)):
            if hasattr(type(self._value), '__le__'):
                return self._value.__le__(other._value)
        
        return NotImplemented
    
    def __ge__(self, other):
        """Delegate to greater-or-equal comparison for the wrapped object."""
        logging.info(f'{self!r}.__ge__({other!r})')
        
        if isinstance(other, type(self)):
            if hasattr(type(self._value), '__ge__'):
                return self._value.__ge__(other._value)
        
        return NotImplemented
    
    def __hash__(self):
        """Delegate to the wrapped object's __hash__ and log the call."""
        logging.info(f'{self!r}.__hash__()')
    
    @property
    def value(self):
        """The value this Noisy wrapper holds."""
        return self._value

In [3]:
logging.getLogger().setLevel(logging.INFO)

In [4]:
a = [*(Noisy(i) for i in range(1, 6)), Noisy(8)]
b = [*(Noisy(i) for i in range(1, 6)), Noisy(9)]

In [5]:
a

[Noisy(1), Noisy(2), Noisy(3), Noisy(4), Noisy(5), Noisy(8)]

In [6]:
b

[Noisy(1), Noisy(2), Noisy(3), Noisy(4), Noisy(5), Noisy(9)]

### Lists&lsquo; `__eq__` and `__ne__` use elements&lsquo; `__eq__`.

No surprise here.

In [7]:
a == b # Uses __eq__. Anything different would be astonishing.

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))


False

In [8]:
a != b  # Also uses __eq__, but that's still fine and unsurprising.

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))


True

In [9]:
a.__eq__(b)  # Same result as with == when we call __eq__ directly.

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))


False

In [10]:
a.__ne__(b)  # Same result as with != when we call __ne__ directly.

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))


True

### Lists&rsquo; `__lt__`, `__gt__`, `__le__`, `__ge__` use elements&rsquo; `__eq__`!

Lexicographical order comparisons use `==` until they find differing elements, or until at least one sequence is exhausted, then they use the order-comparison operator you used to find out which direction the disparity is in.

This is the case both for the strict (`<` and `>`) and non-strict (`<=` and `>=`) order comparison operators.

In [11]:
a < b

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))
INFO:root:Noisy(8).__lt__(Noisy(9))


True

In [12]:
a > b

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))
INFO:root:Noisy(8).__gt__(Noisy(9))


False

In [13]:
a <= b

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))
INFO:root:Noisy(8).__le__(Noisy(9))


True

In [14]:
a >= b

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))
INFO:root:Noisy(8).__ge__(Noisy(9))


False

In [15]:
a.__lt__(b)  # Same result as with < when we call __lt__ directly.

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))
INFO:root:Noisy(8).__lt__(Noisy(9))


True

In [16]:
a.__gt__(b)  # Same result as with > when we call __gt__ directly.

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))
INFO:root:Noisy(8).__gt__(Noisy(9))


False

In [17]:
a.__le__(b)  # Same result as with <= when we call __le__ directly.

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))
INFO:root:Noisy(8).__le__(Noisy(9))


True

In [18]:
a.__ge__(b)  # Same result as with <= when we call __ge__ directly.

INFO:root:Noisy(1).__eq__(Noisy(1))
INFO:root:Noisy(2).__eq__(Noisy(2))
INFO:root:Noisy(3).__eq__(Noisy(3))
INFO:root:Noisy(4).__eq__(Noisy(4))
INFO:root:Noisy(5).__eq__(Noisy(5))
INFO:root:Noisy(8).__eq__(Noisy(9))
INFO:root:Noisy(8).__ge__(Noisy(9))


False

### So you can&rsquo;t use it with arbitrary weak orderings.

Lexicographical order comparison in Python uses equality comparison to find the first position (if any) where objects differ, only then doing the order comparison. This always works with total orderings, but it would not usually work with non-total weak orderings.

In [19]:
from compare import OrderIndistinct

In [20]:
c = [OrderIndistinct('C'), 10]
d = [OrderIndistinct('D'), 20]

In [21]:
c < d  # We would want this to be True, since 10 < 20.

False

In [22]:
c > d  # False, but not for the reason we want it to be.

False

The second element never got compared, as this reveals:

In [23]:
cc = [OrderIndistinct('CC'), Noisy(10)]
dd = [OrderIndistinct('DD'), Noisy(20)]

In [24]:
c < d

False

In [25]:
c > d

False

`Noisy` didn&rsquo;t log anything, so in neither case were the second elements compared.